<a href="https://colab.research.google.com/github/rizky-gumelar/Fall-Detection-Classification/blob/main/CreateSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q mediapipe
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 43.7 MB/s eta 0:00:00


In [ ]:
#@markdown To better demonstrate the Pose Landmarker API, we have created a set of visualization tools that will be used in this colab. These will draw the landmarks on a detect person, as well as the expected connections between those markers.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [ ]:
import os
import zipfile

zip_path = '/content/drive/MyDrive/DataSlayer/multiclass_data.zip'
extract_path = '/'  # Tempat untuk mengekstrak file

# Ekstraksi file ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Cek apakah dataset berhasil diekstrak
os.listdir(extract_path)

['libx32',
 'bin',
 'mnt',
 'var',
 'usr',
 'sbin',
 'srv',
 'home',
 'media',
 'sys',
 'boot',
 'run',
 'lib32',
 'lib',
 'opt',
 'root',
 'etc',
 'proc',
 'dev',
 'tmp',
 'lib64',
 'content',
 '.dockerenv',
 'tools',
 'datalab',
 'python-apt',
 'python-apt.tar.xz',
 'NGC-DL-CONTAINER-LICENSE',
 'cuda-keyring_1.0-1_all.deb']

In [ ]:
import os

for root, dirs, files in os.walk('/content/multiclass_data'):
    level = root.replace('/content/multiclass_data', '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}- {os.path.basename(root)}/")  # Cetak hanya folder

- multiclass_data/
  - forward_falls/
  - picking/
  - backward_falls/
  - laying/
  - jumping/
  - standing_falls/
  - walking/
  - stretching/
  - squat/
  - sitting_falls/
  - left_falls/
  - right_falls/


In [ ]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import numpy as np
import os

kelas = [
    'forward_falls', 'right_falls', 'left_falls', 'backward_falls',
    'standing_falls', 'sitting_falls', 'walking', 'jumping',
    'stretching', 'squat', 'picking', 'laying'
]

# STEP 2: Create a PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# STEP 3: Define the input folder and output folder.
input_folder = "/content/multiclass_data/"
output_folder = "/content/multiclass_data_masks/"

# Create output directories if not exist
for category in kelas:
  os.makedirs(os.path.join(output_folder, category), exist_ok=True)

# STEP 4: Process images in 'fall/' and 'non_fall/' directories.
for category in kelas:
    input_category_folder = os.path.join(input_folder, category)
    output_category_folder = os.path.join(output_folder, category)

    for filename in os.listdir(input_category_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            input_image_path = os.path.join(input_category_folder, filename)
            output_image_path = os.path.join(output_category_folder, filename)

            # Load the input image.
            image = mp.Image.create_from_file(input_image_path)

            # Detect pose landmarks from the input image.
            detection_result = detector.detect(image)

            # Check if segmentation masks are available.
            if detection_result.segmentation_masks:
                # Generate the segmentation mask.
                segmentation_mask = detection_result.segmentation_masks[0].numpy_view()
                visualized_mask = np.repeat(segmentation_mask[:, :, np.newaxis], 3, axis=2) * 255

                # Save the visualized mask as an image.
                visualized_mask = visualized_mask.astype(np.uint8)  # Ensure the data type is uint8.
                cv2.imwrite(output_image_path, visualized_mask)
            else:
                print(f"No segmentation mask for {filename}")

print("Segmentation masks have been saved successfully.")


No segmentation mask for frame011_subject-2.jpg
No segmentation mask for frame013_subject-2.jpg
No segmentation mask for frame033_subject-1.jpg
No segmentation mask for frame025_subject-1.jpg
No segmentation mask for frame044_subject-4.jpg
No segmentation mask for frame052_subject-4.jpg
No segmentation mask for frame039_subject-1.jpg
No segmentation mask for frame008_subject-2.jpg
No segmentation mask for frame007_subject-2.jpg
No segmentation mask for frame019_subject-1.jpg
No segmentation mask for frame010_subject-3.jpg
No segmentation mask for frame018_subject-1.jpg
No segmentation mask for frame020_subject-1.jpg
No segmentation mask for frame024_subject-2.jpg
No segmentation mask for frame027_subject-2.jpg
No segmentation mask for frame001_subject-4.jpg
No segmentation mask for frame012_subject-4.jpg
No segmentation mask for frame086_subject-4.jpg
No segmentation mask for frame099_subject-1.jpg
No segmentation mask for frame048_subject-3.jpg
No segmentation mask for frame116_subjec

In [ ]:
# @title Total gambar
import os

def count_images_in_directory(directory, valid_formats=('.jpg', '.jpeg', '.png', '.bmp')):
    """
    Menghitung total jumlah gambar dalam sebuah folder termasuk subfoldernya.

    Args:
        directory (str): Path direktori utama.
        valid_formats (tuple): Format file gambar yang valid.

    Returns:
        int: Total jumlah file gambar.
    """
    total_images = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(valid_formats):  # Periksa format file
                total_images += 1
    return total_images

# Contoh penggunaan
folder_path = "/content/testdata_mask" # @param {"type":"string"}
total_images = count_images_in_directory(folder_path)
print(f"Total gambar dalam folder '{folder_path}': {total_images}")


Total gambar dalam folder '/content/testdata_mask': 2152


In [ ]:
# Tentukan folder yang ingin di-zip
folder_to_zip = '/content/testdata_mask'  # Ganti 'folder_name' dengan nama folder Anda
output_zip_file = 'testdata_mask.zip'  # Nama file ZIP yang akan dibuat

# Pindah ke direktori induk dari folder yang akan di-zip
os.chdir('/content')  # Pindah ke direktori '/content'

# Perintah zip untuk membuat file ZIP
!zip -r "{output_zip_file}" "{folder_to_zip}"

print(f"Folder '{folder_to_zip}' telah berhasil di-zip menjadi '{output_zip_file}'.")

  adding: content/testdata_mask/ (stored 0%)
  adding: content/testdata_mask/f949cd3b02.jpg (deflated 56%)
  adding: content/testdata_mask/fa7efaa482.jpg (deflated 55%)
  adding: content/testdata_mask/0fc0f3fe24.jpg (deflated 55%)
  adding: content/testdata_mask/2297123d87.jpg (deflated 59%)
  adding: content/testdata_mask/aaa23e214a.jpg (deflated 61%)
  adding: content/testdata_mask/c515d7057a.jpg (deflated 62%)
  adding: content/testdata_mask/42250d6d18.jpg (deflated 60%)
  adding: content/testdata_mask/a4ad62c721.jpg (deflated 65%)
  adding: content/testdata_mask/8db8680806.jpg (deflated 65%)
  adding: content/testdata_mask/b3857db71a.jpg (deflated 71%)
  adding: content/testdata_mask/dd0a309a5f.jpg (deflated 51%)
  adding: content/testdata_mask/244a426a24.jpg (deflated 64%)
  adding: content/testdata_mask/ae8c20ce56.jpg (deflated 60%)
  adding: content/testdata_mask/6b9fa95e31.jpg (deflated 63%)
  adding: content/testdata_mask/c70fc39660.jpg (deflated 71%)
  adding: content/testdat

In [ ]:
# Path file sumber di Colab
source = "/content/testdata_mask.zip"

# Path tujuan di Google Drive
destination = "/content/drive/MyDrive/DataSlayer"

# Copy file
!cp "$source" "$destination"

print("File berhasil dicopy ke:", destination)

File berhasil dicopy ke: /content/drive/MyDrive/DataSlayer


In [ ]:
import os
import zipfile
zip_path = '/content/drive/MyDrive/DataSlayer/data-slayer-2-0-machine-learning-competition.zip'
extract_path = '/content/dataset'  # Tempat untuk mengekstrak file

# Ekstraksi file ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Cek apakah dataset berhasil diekstrak
os.listdir(extract_path)

['train', 'test', 'sample_submission.csv']

In [ ]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import numpy as np
import os
import shutil

kelas = [
    'forward_falls', 'right_falls', 'left_falls', 'backward_falls',
    'standing_falls', 'sitting_falls', 'walking', 'jumping',
    'stretching', 'squat', 'picking', 'laying'
]

# STEP 2: Create a PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# STEP 3: Define the input folder and output folder.

input_category_folder = "/content/dataset/test"
output_category_folder = "/content/testdata_mask"

os.makedirs(output_category_folder, exist_ok=True)

for filename in os.listdir(input_category_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        input_image_path = os.path.join(input_category_folder, filename)
        output_image_path = os.path.join(output_category_folder, filename)

        # Load the input image.
        image = mp.Image.create_from_file(input_image_path)

        # Detect pose landmarks from the input image.
        detection_result = detector.detect(image)

        # Check if segmentation masks are available.
        if detection_result.segmentation_masks:
            # Generate the segmentation mask.
            segmentation_mask = detection_result.segmentation_masks[0].numpy_view()
            visualized_mask = np.repeat(segmentation_mask[:, :, np.newaxis], 3, axis=2) * 255

            # Save the visualized mask as an image.
            visualized_mask = visualized_mask.astype(np.uint8)  # Ensure the data type is uint8.
            cv2.imwrite(output_image_path, visualized_mask)
        else:
            shutil.copy(input_image_path, output_image_path)
            print(f"No segmentation mask for {filename}")

print("Segmentation masks have been saved successfully.")


No segmentation mask for 61a25ffedb.jpg
No segmentation mask for f6a5915841.jpg
No segmentation mask for 5c8e48d4ea.jpg
No segmentation mask for 4d33dc243b.jpg
No segmentation mask for be01612c13.jpg
No segmentation mask for 599e2f535a.jpg
No segmentation mask for 8fbe0640cf.jpg
No segmentation mask for 1c97eeb983.jpg
No segmentation mask for ad34e149ae.jpg
No segmentation mask for 99a1af480f.jpg
No segmentation mask for 4b6287d75a.jpg
No segmentation mask for 3c69cee37f.jpg
No segmentation mask for 68d361ac58.jpg
No segmentation mask for a082c58398.jpg
No segmentation mask for 843567a105.jpg
No segmentation mask for e4190f0ca6.jpg
No segmentation mask for fd1a71a278.jpg
No segmentation mask for 6f1077d4cd.jpg
No segmentation mask for c010159f8b.jpg
No segmentation mask for 816da92cdd.jpg
No segmentation mask for 9bc7dbc33b.jpg
No segmentation mask for 6382950826.jpg
No segmentation mask for 2f9eda3605.jpg
No segmentation mask for bb2c7b684d.jpg
No segmentation mask for dd195dbc9e.jpg
